# Watson Assistant - Blind Testing
### Omar Megawer 
#### 25-April-2018

In [1]:
import pandas as pd
from watson_developer_cloud import AssistantV1
import itertools
import sys
import openpyxl

#### Read Blind Set csv file
The csv file shouldn't have any headers.

In [2]:
blind_set = pd.read_csv('blinds.csv', encoding='utf8', names=['Question','Intent'])

#### Watson Assistant Credentials

In [8]:
AssistantCred = {
    'url':"https://gateway.watsonplatform.net/conversation/api/",
    'workspace_id':'',
    'user':'',
    'password':''
}

In [9]:
assistant = AssistantV1(version='2018-02-16', url=AssistantCred.get('url'), username=AssistantCred.get('user'), password=AssistantCred.get('password'))

---

#### Progress Bar Function

In [10]:
def progress(count, total, suffix=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s .%s\r' % (bar, percents, '%', suffix))
    sys.stdout.flush()  

---

#### Preparing Results Dataframe 
This is currently set to record till R@5

In [11]:
full_results = pd.DataFrame(columns=[
            'Question','Expected Intent']+
                            list(itertools.chain(*[['Intent{}'.format(i),
                                                    'Confidence{}'.format(i),
                                                    'R@{}'.format(i)]
                                      for i in range(1, 5+1)])))

#### Sending Questions to Watson Assistant

In [12]:
count = 0
numberOfQuestions = len(blind_set.index)
print("Blind Set has", numberOfQuestions , "questions")

for index, row in blind_set.iterrows():
            
            question = row['Question']
            expected_intent = row['Intent']
            
            message = { 'text': question }
            
            progress(count,numberOfQuestions, '' )
            
            response = assistant.message(workspace_id=AssistantCred.get('workspace_id'), input=message, alternate_intents=True)
            
            rec = { 'Question': question, 'Expected Intent': expected_intent }
            
            intents = response['intents']
            
            for i in range(0,len(intents)):
                rec.update({
                    'Intent{}'.format(i+1): intents[i]['intent'],
                    'Confidence{}'.format(i+1): intents[i]['confidence'],
                    'R@{}'.format(i+1): 0
                    })

            for i in range(len(intents) -1, -1, -1):
                if rec['Expected Intent'] == rec['Intent{}'.format(i+1)]:
                    rec['R@{}'.format(i+1)] = 1
                    
            full_results = full_results.append(rec,ignore_index=True)
            count = count + 1
progress(numberOfQuestions,numberOfQuestions, 'Test is Done' )


Blind Set has 5 questions


---

#### Prepare Detailed Result Data Frame

In [13]:
full_results = full_results.fillna(0)

results = pd.DataFrame(columns=['Question', 'Expected Intent', 'Recall@', 'Found', 'Confidence'])
        
for index, row in full_results.iterrows():  
    
    record = {'Question': row['Question'], 'Expected Intent': row['Expected Intent'] }
    
    for i in range(0,5): 
        record.update({
            'Recall@': i+1,
            'Confidence': row['Confidence{}'.format(i+1)],
            'Found': row['R@{}'.format(i+1)] == 1
        })
        results = results.append(record,ignore_index=True)

---

#### Calculations

This will calculate the Recall@1 for more details check the Excel Report.

In [14]:
precisionDF = results.query('Found == True') 
precisionDF
precision = (len(precisionDF.index) / numberOfQuestions) * 100
accuracyDF = precisionDF[precisionDF['Recall@'] == 1]
accuracy = (len(accuracyDF.index) / numberOfQuestions) * 100
print("The accuracy for the set is: ",accuracy)

The accuracy for the set is:  0.0


#### Output Results Report to Excel

In [15]:
writer = pd.ExcelWriter('Report.xlsx')
results.to_excel(writer, "Blind Test", encoding='UTF8')
writer.save()